# Spatially Enabled Data Frames

<img src="./img/title_image.jpg"/>

## Overview 

- Pandas data frames 
- Custom namespace on Series and DataFrame objects
    + `geom` on Series
    
    ```python
    >>> df['SHAPE'].geom.area
    ```
    
    + `spatial` on the the DataFrame
    
    ```python
    >>> df.spatial.project(4326)
    ```

- Cross platform spatial analysis
    + Mac, Linux, and Windows

- Multi-geometry engine support

    + Esri Arcpy's Engine
    + Shapely/Geos
    
- Read/write data

    + Fiona, shapefile, and arcpy

## Getting Started

It starts with two imports

In [ ]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

- Loads Pandas
- The `GeoAccessor` and `GeoSeriesAccessor` load the namespaces into Pandas

### Data I/O

#### Consumption

- Feature Layers
- Data Frames
- Feature classes

```python

    from arcgis.features import FeatureLayer
    fl = FeatureLayer(("https://services2.arcgis.com/zPFLSOZ5HzUzzTQb/arcgis"
                       "/rest/services/CensusBlockGroup/FeatureServer/0"))
    sdf1 = pd.DataFrame.spatial.from_layer(fl)
    sdf2 = pd.DataFrame.spatial.from_featureclass("./data/historic_traffic.shp")
    df_earthquakes = pd.read_csv("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.csv")
    sdf3 = pd.DataFrame.spatial.from_xy(df=df_earthquakes, 
                                        x_column="longitude", 
                                        y_column="latitude", 
                                        sr=4326)
```

#### Persistence of Data

- Tables
- Feature Classes
- Services

```python
    item = gis.content.import_data(df=sdf1, title="CensusBlockGroup")
    sdf1.spatial.to_featureclass(location="./data/stage.gdb/census")
```

## Mapping

- Provides a rich visualization of data
- Data does not have to exist on an Enterprise
- Map both local and service data

### Simple Rendering of Data

- Simple renders can be circles, squares, solid colors, etc...
- Single color only

In [ ]:
from arcgis.gis import GIS
gis = GIS("https://slustl.maps.arcgis.com/", "gregbrunner_slugis")

In [ ]:
first_map = gis.map('New York, NY')
first_map.basemap = 'streets'
first_map

In [ ]:
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
flayer = item.layers[0]
df = flayer.query(where="AGE_45_54 < 1500", as_df=True)

In [ ]:
df.head()

In [ ]:
df.spatial.plot(map_widget=first_map)

### Advanced Simple Rendering


In [ ]:
m2 = gis.map('United States')
m2.zoom = 3
m2

In [ ]:
m2.center = [39,-98]
df.spatial.plot(map_widget=m2,
            symbol_type='simple',
            symbol_style='d', # d - for diamonds
            colors='Reds_r',
            cstep=10,
            outline_color='Blues',
            marker_size=10)

### Class Break Renderer

In [ ]:
m3 = gis.map('Reno, NV', zoomlevel=4)
m3.center = [39,-98]
m3

In [ ]:
df.spatial.plot(map_widget = m3,
                renderer_type='u', # specify the unique value renderer using its notation 'u'
                col='ST'  # column to get unique values from
               )

### Class Break Renderer


In [ ]:
m4 = gis.map('Reno, NV', zoomlevel=4)
m4.center = [39,-98]
m4

In [ ]:
df.spatial.plot(map_widget=m4,
               renderer_type='c',  # for class breaks renderer
               method='esriClassifyNaturalBreaks',  # classification algorithm
               class_count=20,  # choose the number of classes
               col='POPULATION',  # numeric column to classify
               cmap='gnuplot2_r',  # color map to pick colors from for each class
               alpha=1  # specify opacity
               )

In [ ]:
m4.legend = True

### Rendering Polygon Example


In [ ]:
from arcgis.features import FeatureLayer
fl = FeatureLayer("https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Counties_Generalized/FeatureServer/0")
#("https://sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2")
county_sdf = fl.query("STATE_NAME='Washington'", out_sr=4326, as_df=True)
county_sdf.head()

In [ ]:
m5 = gis.map('Seattle, WA', zoomlevel=6)
m5.basemap = 'dark-gray-vector'
m5

In [ ]:
county_sdf.spatial.plot(map_widget=m5,
                        renderer_type='c',  
                        method='esriClassifyNaturalBreaks',  
                        class_count=7,  
                        col='AGE_20_24', 
                        cmap='RdPu',  
                        alpha=0.8,
                        line_width=.25)

#### Help Functions

- How do we know what colormaps to use?

In [ ]:
from arcgis.mapping import display_colormaps
display_colormaps()

## Spatial Index

- Quickly find spatial locations 
    + generalized locations
- Based on Minimum bounding rectangles

### Index Visualized

<img src="./img/spatial_index_viz.png" />

### Spatial Index Example

In [ ]:
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
item

In [ ]:
sdf = item.layers[0].query(as_df=True, out_sr=4326) # Major US Cities

In [ ]:
index = sdf.spatial.sindex(stype='quadtree')

In [ ]:
nj_cities = index.intersect((-75.55956796790353, 38.928522146813044, 
                             -73.9024505439044, 41.35763612214295))

In [ ]:
m6 = gis.map("New Jersey")
m6

In [ ]:
sdf.iloc[nj_cities].spatial.plot(m6)

#### Notice the Following

- The bounding box returns locations outside of New Jersey
    + This means the the selction by location is **generalized**
    

## Spatial Analysis

<img src="./img/analysis_china.jpg"/>

- We have data, but how to we gain insights into the data?
- Python API provides a wealth of vector based analytics

### Using Geoprocessing Tools

- The work horse of analysis
- Service provide tools not available in the standard libary

In [ ]:
from arcgis import create_viewshed
sub_sdf = sdf.iloc[[2981, 2982, 2983, 2984]]

In [ ]:
vs = create_viewshed(input_layer=sub_sdf.spatial.to_feature_collection(), 
                     maximum_distance=20, 
                     max_distance_units="Miles")

In [ ]:
m7 = gis.map('South Charleston, West Virginia', zoomlevel=11)
m7.basemap = 'dark-gray-vector'
m7

In [ ]:
m7.add_layer(vs)
sub_sdf.spatial.plot(m7,
                    symbol_type='simple',
                    symbol_style='d', # d - for diamonds
                    colors='Reds_r',
                    cstep=20,
                    outline_color='Blues',
                    marker_size=20)

### Using GeoEnrichment 

<img src="./img/enrichment_talk.jpeg"/>

- Provides insights into a location or area
- Uses credits
- Driven heavily on SeDF

In [ ]:
from arcgis.geoenrichment import enrich
enrich_df = enrich(study_areas=sub_sdf, data_collections=['Age'] )
enrich_df.head()

In [ ]:
m8 = gis.map('South Charleston, West Virginia', zoomlevel=11)
m8.basemap = 'dark-gray'
m8

In [ ]:
enrich_df.spatial.plot(m8)

### On to Custom Analysis

- Besides the built in tool SeDF allows for complete customize of analysis process


In [ ]:
df = pd.read_csv("./data/listings.csv")
df.head()

#### Examining the Review Scores

In [ ]:
ratings = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
          'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
          'review_scores_value', 'latitude', 'longitude']
df[ratings].describe()

In [ ]:
data = df[~df['review_scores_value'].isna()].copy().reset_index(drop=True)
sdf_airbnb = pd.DataFrame.spatial.from_xy(df=data, y_column='latitude', x_column='longitude', sr=4326)

In [ ]:
sdf_airbnb.spatial.full_extent

In [ ]:
m10 = gis.map()
m10.extent = sdf_airbnb.spatial.full_extent
m10

#### K-Means Clustering

- Unsupervised learning  technique
    + finds hidden structures in data when we don't know the answer upfront


In [ ]:
from sklearn import cluster

- Remove and NaN or NULL value from the columns of interest
- Convert price from string to numeric
- Select only 'Entire home/apt' 

In [ ]:
q = (sdf_airbnb['room_type'] == 'Entire home/apt') & (sdf_airbnb['beds'] >=4)

sdf_airbnb = (sdf_airbnb
              .loc[q]
              .dropna(axis=0, subset=ratings)
              .copy()
              .reset_index(drop=True))

sdf_airbnb['RENTALPRICE'] = (sdf_airbnb
                                    .price
                                    .str
                                    .replace("$", "")
                                    .str.replace(",","")
                             .apply(float))

In [ ]:
import matplotlib.pyplot as plt
distortions = []
for i in range(1,11):
    km = cluster.KMeans(n_clusters=i,
           init='k-means++',
           n_init=10,
           max_iter=300,
           random_state=1)
    km.fit(sdf_airbnb[['RENTALPRICE']])
    distortions.append(km.inertia_)
plt.figure(figsize=(10,10))
plt.plot(range(1,11), distortions, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
kmeans5 = cluster.KMeans(n_clusters=4,
                        init='random',
                        n_init=10,
                        max_iter=300,
                        random_state=0)
k5cls = kmeans5.fit_predict(sdf_airbnb[['RENTALPRICE']])
sdf_airbnb['class'] = k5cls

In [ ]:
display_columns = ['SHAPE', 'class', 'name', 'summary', "price"]

In [ ]:
m11 = gis.map()
m11.extent = sdf_airbnb.spatial.full_extent
m11

In [ ]:
sdf_airbnb[display_columns].copy().spatial.plot(map_widget=m11,
               renderer_type='u',  # for unique renderer
               col='class',  # numeric column to classify
               cmap='gist_ncar',  
               line_width=0,
               alpha=0.7  # specify opacity
               )

In [ ]:
m11.legend = True